# Week 1: Find Clusters of Infected People

<span style="color:red">
**URGENT WARNING**

We have been receiving reports from health facilities that a new, fast-spreading virus has been discovered in the population. To prepare our response, we need to understand the geospatial distribution of those who have been infected. Find out whether there are identifiable clusters of infected individuals and where they are.    
</span>

Your goal for this notebook will be to estimate the location of dense geographic clusters of infected people using incoming data from week 1 of the simulated epidemic.

## Imports

In [1]:
import cudf
import cuml

import cupy as cp

## Load Data

Begin by loading the data you've received about week 1 of the outbreak into a cuDF data frame. The data is located at `'./data/week1.csv'`. For this notebook you will only need the `'lat'`, `'long'`, and `'infected'` columns. Either drop the columns after loading, or use the `cudf.read_csv` named argument `usecols` to provide a list of only the columns you need.

In [2]:
gdf = cudf.read_csv('./data/week1.csv', usecols=['lat', 'long', 'infected'])
gdf.head()

 missing cuda symbols while dynamic loading
 cuFile initialization failed


,lat,long,infected
0,54.522510,-1.571896,False
1,54.554030,-1.524968,False
2,54.552486,-1.435203,False
3,54.537189,-1.566215,False
4,54.528212,-1.588462,False


## Make Data Frame of the Infected

Make a new cuDF data frame `infected_df` that contains only the infected members of the population.

In [3]:
infected_df = gdf[gdf['infected'] == True].reset_index()
infected_df.head()

,index,lat,long,infected
0,28928759,54.472766,-1.654932,True
1,28930512,54.529717,-1.667143,True
2,28930904,54.512986,-1.589866,True
3,28932226,54.522322,-1.380694,True
4,28933748,54.541660,-1.613490,True


In [4]:
infected_df.shape

(18148, 4)

## Make Grid Coordinates for Infected Locations

Provided for you in the next cell (which you can expand by clicking on the "..." and contract again after executing by clicking on the blue left border of the cell) is the lat/long to OSGB36 grid coordinates converter you used earlier in the workshop. Use this converter to create grid coordinate values stored in `northing` and `easting` columns of the `infected_df` you created in the last step.

In [5]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (N)
    long: longitude coordinate (E)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000 # northing of true origin
    E0 = 400000 # easting of true origin
    F0 = .9996012717 # scale factor on central meridian
    phi0 = 49 * cp.pi / 180 # latitude of true origin
    lambda0 = -2 * cp.pi / 180 # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

In [6]:
cupy_infected_lat = cp.asarray(infected_df['lat'])
cupy_infected_long = cp.asarray(infected_df['long'])

northing, easting = latlong2osgbgrid_cupy(cupy_infected_lat, cupy_infected_long)

infected_df['northing'] = cudf.Series(northing).astype('float32')
infected_df['easting'] = cudf.Series(easting).astype('float32')

infected_df.head()

,index,lat,long,infected,northing,easting
0,28928759,54.472766,-1.654932,True,508670.06250,422359.7500
1,28930512,54.529717,-1.667143,True,515002.65625,421538.5625
2,28930904,54.512986,-1.589866,True,513167.53125,426549.8750
3,28932226,54.522322,-1.380694,True,514305.28125,440081.2500
4,28933748,54.541660,-1.613490,True,516349.12500,425003.0000


## Find Clusters of Infected People

Use DBSCAN to find clusters of at least 25 infected people where no member is more than 2000m from at least one other cluster member. Create a new column in `infected_df` which contains the cluster to which each infected person belongs.

In [7]:
eps = 4000
dbscan = cuml.DBSCAN(eps=eps)

In [8]:
infected_df['cluster'] = dbscan.fit_predict(infected_df[['northing', 'easting']])
infected_df.sample(5)

,index,lat,long,infected,northing,easting,cluster
6586,32260955,53.472710,-2.108076,True,397364.31250,392827.15625,2
5662,32045723,53.457298,-2.365642,True,395706.71875,375724.03125,2
3316,31407660,53.601508,-2.310311,True,411731.37500,379467.50000,2
17608,41662200,52.591169,-2.009550,True,299302.84375,399353.15625,2
897,30541436,53.272099,-2.244793,True,375071.06250,383676.93750,2


In [9]:
infected_df['cluster'].nunique()

25

## Find the Centroid of Each Cluster

Use grouping to find the mean `northing` and `easting` values for each cluster identified above.

In [10]:
clusters = infected_df[['cluster', 'northing', 'easting']].groupby(['cluster'])
avg_clusters = clusters.mean()
avg_clusters

,northing,easting
cluster,,
22,362953.676471,288606.794118
10,404611.200000,494369.050000
6,452361.724138,461293.551724
11,458582.593750,431573.531250
9,432321.156250,511245.718750
4,465853.500000,392649.050000
23,351050.607143,301848.892857
20,272514.550000,379057.200000
21,266163.725000,394227.400000


In [11]:
clusters.head()

,cluster,northing,easting
0,-1,508670.06250,422359.75000
1,0,515002.65625,421538.56250
2,0,513167.53125,426549.87500
3,-1,514305.28125,440081.25000
4,0,516349.12500,425003.00000
...,...,...,...
17844,22,356904.96875,288171.06250
17848,23,350684.78125,302714.81250
17852,22,363444.93750,287573.46875
17857,23,348289.00000,302890.25000


Find the number of people in each cluster by counting the number of appearances of each cluster's label in the column produced by DBSCAN.

In [12]:
cnt_clusters = clusters.count()
cnt_clusters

,northing,easting
cluster,,
-1,549,549
0,8,8
1,12,12
2,17347,17347
3,5,5
4,5,5
5,29,29
6,58,58
7,5,5


In [13]:
max_easting_row = cnt_clusters[cnt_clusters['easting'] == cnt_clusters['easting'].max()]
max_easting_row

,northing,easting
cluster,,
2,17347,17347


In [19]:
infected_df[infected_df['cluster'] == 2].sort_values(by='easting', ascending=False)

,index,lat,long,infected,northing,easting,cluster
15273,38247343,53.203078,-0.743497,True,368102.40625,483918.21875,2
15406,38974832,53.014003,-0.750882,True,347063.53125,483791.71875,2
15262,38228071,52.980926,-0.789093,True,343340.59375,481290.75000,2
15485,39013069,53.022302,-0.796761,True,347933.96875,480698.78125,2
15494,39019513,53.264826,-0.792025,True,374914.75000,480561.28125,2
...,...,...,...,...,...,...,...
17972,57265113,53.214961,-3.392729,True,369592.59375,307009.87500,2
18008,57275387,53.136868,-3.394249,True,360908.50000,306739.15625,2
17996,57272293,53.196086,-3.399997,True,367502.62500,306483.56250,2
17917,57202983,53.143409,-3.414618,True,361662.84375,305391.15625,2


## Take the Assessment

After completing the work above, visit the *Launch Section* web page that you used to launch this Jupyter Lab. Scroll down below where you launched Jupyter Lab, and answer the question *Week 1 Assessment*. You can view your overall progress in the assessment by visiting the same *Launch Section* page and clicking on the link to the *Progress* page.

There will be additional questions for you to answer after completing the remaining notebooks. On the *Progress* page, if you have successfully answered all the assessment questions, you can click on *Generate Certificate* to receive your certificate in the course.

![launch_task_page](./images/launch_task_page.png)

<div align="center"><h2>Please Restart the Kernel</h2></div>

In [20]:
import IPython
app = IPython.Application.instance()
app.kernel.do_shutdown(True)

{'status': 'ok', 'restart': True}